In [95]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import random
from random import choice,randrange
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import math
import time
from torch_geometric.nn import GCNConv,GraphConv,GATConv,SAGEConv

random_seed=280085

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
from google.colab import drive
drive.mount("/content/drive")
%cd drive/My Drive/possible_paper


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/possible_paper'
/content/drive/My Drive/possible_paper


In [97]:
from utils import *  #In this files are reported the most useful functions

In [98]:
X = torch.load('instance').T      #Instance matrix
A = torch.load('adjacencyCOO')  #Normal adjacency matrix format is obtained with torch.load('adjacency')


In [99]:
num2artist = load_data('dizofartist.json')
artist2num = {num2artist[key]:key for key in num2artist}

## Import the data with Torch geometric:


In [102]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import train_test_split_edges

data = Data(x=X, edge_index = A)
loader = DataLoader([data], batch_size=512, shuffle=True)